In [ ]:
"""
NBA Offseason Grading System: Calculate Team Offseason Grades

Goal: Calculates offseason grades (A+ to F) for all 30 NBA teams based on
    their player acquisitions, losses, and retentions.

What the script does:
    1. Calculate fit scores for every player (Role Fit + Skill Fit)
    2. Weights fit scores by playing time and role importance
    3. Sums weighted fit scores of all gains, losses, and retained players
    4. Calculates net impact
    5. Assigns letter grade 


Input file is the standardized Excel file
    - Contains 4 sheets with z-scored player and team metrics

Output file will be a new excel file with the offseason grades and team scores
"""

# External libraries needed for this script
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# File paths
standardized_file = 'data/processed/Offseason_Grades_Standardized.xlsx'
output_file = 'output/Offseason_Grades_Output_V2.xlsx'

# Script begins
print("="*80)
print("NBA OFFSEASON GRADING SYSTEM - VERSION 2 (ENHANCED ROLE FIT)")
print("="*80)
print("\nLoading standardized data...")


# Load data from each sheet
player_role = pd.read_excel(standardized_file, sheet_name='Player Role Fit')
player_skill = pd.read_excel(standardized_file, sheet_name='Player Skill Fit')
team_role = pd.read_excel(standardized_file, sheet_name='Team Role Fit')
team_skill = pd.read_excel(standardized_file, sheet_name='Team Skill Fit')

print("✓ Data loaded successfully!")

# Combine all player information into one DataFrame
players = player_role.merge(player_skill, on='Player Code', how='inner', suffixes=('_role', '_skill'))

print(f"✓ Merged player data: {players.shape[0]} players")

# After merge, some columns exist twice, I keep the _role versions
if '2024-2025 Team_role' in players.columns:
    players['2024-2025 Team'] = players['2024-2025 Team_role']
if '2025-2026 Team_role' in players.columns:
    players['2025-2026 Team'] = players['2025-2026 Team_role']
if 'Transaction_role' in players.columns:
    players['Transaction'] = players['Transaction_role']
if 'Player_role' in players.columns:
    players['Player'] = players['Player_role']
if 'Defensive Role_role' in players.columns:
    players['Defensive Role'] = players['Defensive Role_role']
if 'Offensive Role_role' in players.columns:
    players['Offensive Role'] = players['Offensive Role_role']

# 3-letter codes to full team names
team_mapping = {
    'ATL': 'Atlanta Hawks', 'BOS': 'Boston Celtics', 'BRK': 'Brooklyn Nets', 'BKN': 'Brooklyn Nets',
    'CHA': 'Charlotte Hornets', 'CHO': 'Charlotte Hornets', 'CHI': 'Chicago Bulls', 'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks', 'DEN': 'Denver Nuggets', 'DET': 'Detroit Pistons', 'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets', 'IND': 'Indiana Pacers', 'LAC': 'LA Clippers', 'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies', 'MIA': 'Miami Heat', 'MIL': 'Milwaukee Bucks', 'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans', 'NYK': 'New York Knicks', 'OKC': 'Oklahoma City Thunder', 'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers', 'PHO': 'Phoenix Suns', 'POR': 'Portland Trail Blazers', 'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs', 'TOR': 'Toronto Raptors', 'UTA': 'Utah Jazz', 'WAS': 'Washington Wizards'
}

players['2024-2025 Team Full'] = players['2024-2025 Team'].map(team_mapping)
players['2025-2026 Team Full'] = players['2025-2026 Team'].map(team_mapping)

print("✓ Team name mapping applied")

# Z-score to (0-1 scale)
def z_score_to_percentile(z):
    if pd.isna(z):
        return 0.5
    return stats.norm.cdf(z)

# Alignment between player and team metric    
def calculate_alignment_score(player_val, team_val):
    if pd.isna(player_val) or pd.isna(team_val):
        return 0.5
    diff = abs(player_val - team_val)
    alignment = np.exp(-diff)
    return alignment

# Playing time multipler based on role
def get_starter_bench_multiplier(role):
    if pd.isna(role):
        return 0.15
    role = str(role).strip()
    if 'Core Starter' in role:
        return 1.0
    elif 'Rotation Starter' in role:
        return 0.80
    elif 'High-Impact Bench' in role:
        return 0.60
    elif 'Rotation Bench' in role:
        return 0.35
    else:
        return 0.15


# Role Fit Calculation
print("\n" + "="*80)
print("STEP 1: Calculating ENHANCED Role Fit Scores")
print("="*80)
print("\nNew Formula: 30% Pace + 30% PORT + 20% Shooting + 20% Usage")

# Role score (0-100)
def calculate_enhanced_role_fit(player_row, team_row):    
    # Pace (30%)
    player_pace = player_row.get('Pace_std')
    team_pace = team_row.get('Pace_std')
    pace_alignment = calculate_alignment_score(player_pace, team_pace)
    
    # Portability (30%)
    port_val = player_row.get('PORT_std')
    port_score = z_score_to_percentile(port_val) if pd.notna(port_val) else 0.5
    
    # Shooting (20%)
    player_3par = player_row.get('r3PAr_std')
    
    # Team's Shooting
    corner_3_fga = team_row.get('Corner 3 FGA')
    atb_3_fga = team_row.get('Above the Break 3 FGA')
    total_fga = (team_row.get('Restricted Area FGA', 0) + 
                 team_row.get('In the Paint FGA', 0) + 
                 team_row.get('Mid Range FGA', 0) + 
                 corner_3_fga + atb_3_fga)
    
    if pd.notna(corner_3_fga) and pd.notna(atb_3_fga) and total_fga > 0:
        team_3p_freq = (corner_3_fga + atb_3_fga) / total_fga
        
        # Normalize team shooting
        team_3p_normalized = team_3p_freq * 2.5
        shooting_alignment = calculate_alignment_score(player_3par, team_3p_normalized)
    else:
        shooting_alignment = 0.5
    
    # Usage (20%)
    player_usg = player_row.get('Usage Rate (USG%)_std')
    usage_quality = z_score_to_percentile(player_usg) if pd.notna(player_usg) else 0.5
    
    # Combine all components
    role_fit_raw = (0.30 * pace_alignment + 
                    0.30 * port_score + 
                    0.20 * shooting_alignment + 
                    0.20 * usage_quality)
    
    role_fit_score = role_fit_raw * 100
    
    return role_fit_score
    
# Skill fit
print("\n" + "="*80)
print("STEP 2: Calculating Skill Fit Scores (UNCHANGED from V1)")
print("="*80)

def calculate_skill_fit_score(player_row):
    """Skill Fit Score - IDENTICAL to Version 1"""
    
    # Offense (50% weight)
    playtype_metrics = [
        'Isolation PPP_std', 'Transition PPP_std', 'PnR Ball Handler PPP_std',
        'PnR Roll Man PPP_std', 'Post Up PPP_std', 'Spot Up PPP_std',
        'Handoff PPP_std', 'Cut PPP_std', 'Off Screen PPP_std'
    ]
    playtype_scores = [z_score_to_percentile(player_row.get(m)) for m in playtype_metrics if pd.notna(player_row.get(m))]
    playtype_production = np.mean(playtype_scores) if playtype_scores else 0.5
    
    shooting_metrics = [
        ('rTS%_std', 0.30), ('SQ_std', 0.30),
        ('Catch & Shoot (3P%)_std', 0.25), ('Pull Up Shooting (eFG%)_std', 0.15)
    ]
    shooting_score = sum(z_score_to_percentile(player_row.get(m)) * w 
                        for m, w in shooting_metrics if pd.notna(player_row.get(m)))
    shooting_count = sum(w for m, w in shooting_metrics if pd.notna(player_row.get(m)))
    shooting_production = shooting_score / shooting_count if shooting_count > 0 else 0.5
    
    playmaking_metrics = [('BC_std', 0.50), ('PR_std', 0.50)]
    playmaking_score = sum(z_score_to_percentile(player_row.get(m)) * w 
                          for m, w in playmaking_metrics if pd.notna(player_row.get(m)))
    playmaking_count = sum(w for m, w in playmaking_metrics if pd.notna(player_row.get(m)))
    playmaking_production = playmaking_score / playmaking_count if playmaking_count > 0 else 0.5
    
    olebron_production = z_score_to_percentile(player_row.get('OLEBRON_std')) if pd.notna(player_row.get('OLEBRON_std')) else 0.5
    
    off_production = (0.40 * playtype_production + 0.30 * shooting_production + 
                     0.15 * playmaking_production + 0.15 * olebron_production)
    
    # Defense (50% weight)
    rim_metrics = [('LT_06_%_std', 0.50), ('PLUSMINUS at Rim_std', 0.50)]
    rim_score = sum(z_score_to_percentile(player_row.get(m)) * w 
                   for m, w in rim_metrics if pd.notna(player_row.get(m)))
    rim_count = sum(w for m, w in rim_metrics if pd.notna(player_row.get(m)))
    rim_protection = rim_score / rim_count if rim_count > 0 else 0.5
    
    activity_metrics = [
        ('raDTOV_std', 0.50), ('Deflections per Game_std', 0.30), ('Contested Shots per Game_std', 0.20)
    ]
    activity_score = sum(z_score_to_percentile(player_row.get(m)) * w 
                        for m, w in activity_metrics if pd.notna(player_row.get(m)))
    activity_count = sum(w for m, w in activity_metrics if pd.notna(player_row.get(m)))
    activity_production = activity_score / activity_count if activity_count > 0 else 0.5
    
    drdb_production = z_score_to_percentile(player_row.get('raDRDB_std')) if pd.notna(player_row.get('raDRDB_std')) else 0.5
    dlebron_production = z_score_to_percentile(player_row.get('DLEBRON_std')) if pd.notna(player_row.get('DLEBRON_std')) else 0.5
    
    def_production = (0.25 * rim_protection + 0.25 * activity_production + 
                     0.15 * drdb_production + 0.35 * dlebron_production)
    
    skill_fit_raw = (0.50 * off_production) + (0.50 * def_production)
    skill_fit_score = skill_fit_raw * 100
    
    return skill_fit_score

def calculate_physical_profile(player_row):
    """Physical Profile - UNCHANGED"""
    length_val = player_row.get('Length_std')
    possize_val = player_row.get('PosSize_numeric_std')
    
    length_percentile = z_score_to_percentile(length_val) if pd.notna(length_val) else 0.5
    possize_percentile = z_score_to_percentile(possize_val) if pd.notna(possize_val) else 0.5
    
    physical_profile = (0.50 * length_percentile + 0.50 * possize_percentile) * 100
    return physical_profile

# Fit score calculations
print("\nCalculating comprehensive fit scores for all players...")

player_fit_scores = []

for idx, player in players.iterrows():
    player_name = player.get('Player')
    player_code = player.get('Player Code')
    team_2024 = player.get('2024-2025 Team')
    team_2025 = player.get('2025-2026 Team')
    team_2024_full = player.get('2024-2025 Team Full')
    
    team_data = team_role[team_role['TEAM'] == team_2024_full]
    
    if team_data.empty:
        print(f"⚠ No team data found for {team_2024_full} ({team_2024}), skipping {player_name}")
        continue
    
    team_row = team_data.iloc[0]
    
    # Calculate scores
    role_fit = calculate_enhanced_role_fit(player, team_row)
    skill_fit = calculate_skill_fit_score(player)
    physical_profile = calculate_physical_profile(player)
    
    # Combined fit = 40% Role, 60% Skill
    combined_fit = (0.40 * role_fit) + (0.60 * skill_fit)
    
    # Weight by playing time
    mpg = player.get('Minutes per Game')
    mpg_weight = (mpg / 48) if pd.notna(mpg) else 0.1
    
    role = player.get('Starter or Bench')
    role_multiplier = get_starter_bench_multiplier(role)
    
    weighted_fit = combined_fit * mpg_weight * role_multiplier
    
    transaction = player.get('Transaction')
    defensive_role = player.get('Defensive Role')
    offensive_role = player.get('Offensive Role')
    
    player_fit_scores.append({
        'Player': player_name,
        'Player Code': player_code,
        '2024-2025 Team': team_2024,
        '2025-2026 Team': team_2025,
        'Transaction': transaction,
        'Role': role,
        'Offensive Role': offensive_role,
        'Defensive Role': defensive_role,
        'MPG': mpg,
        'Role Fit Score': role_fit,
        'Skill Fit Score': skill_fit,
        'Combined Fit Score': combined_fit,
        'Physical Profile': physical_profile,
        'MPG Weight': mpg_weight,
        'Role Multiplier': role_multiplier,
        'Weighted Fit': weighted_fit
    })

fit_df = pd.DataFrame(player_fit_scores)
print(f"✓ Calculated enhanced fit scores for {len(fit_df)} players")

# Flag two-way players
fit_df['Is_Two_Way'] = fit_df['Transaction'].isin(['Re-sign (TW)', 'Sign (TW)'])
fit_df['Is_Rookie'] = False  # Will be set for rookies later


# Identify gains, losses, retained players
def identify_transactions(fit_df, team_code):
    scoring_df = fit_df[(fit_df['Is_Two_Way'] == False) & (fit_df['Is_Rookie'] == False)].copy()
    
    gains = scoring_df[
        (scoring_df['2025-2026 Team'] == team_code) &
        (scoring_df['2024-2025 Team'] != team_code) &
        (~scoring_df['Transaction'].isin(['Re-sign', 'Re-sign (TW)', 'Re-sign (Extension)', np.nan]))
    ].copy()
    
    losses = scoring_df[
        (scoring_df['2024-2025 Team'] == team_code) &
        (scoring_df['2025-2026 Team'] != team_code) &
        (
            ((scoring_df['Transaction'].notna()) & (scoring_df['Transaction'] != 'None')) |
            (scoring_df['2025-2026 Team'] == 'FA')
        )
    ].copy()
    
    retained = scoring_df[
        (scoring_df['2024-2025 Team'] == team_code) &
        (scoring_df['2025-2026 Team'] == team_code) &
        (scoring_df['Transaction'].isin(['Re-sign', 'Re-sign (TW)', 'Re-sign (Extension)']))
    ].copy()
    
    return gains, losses, retained

# Get all teams
all_teams = pd.concat([fit_df['2024-2025 Team'], fit_df['2025-2026 Team']]).unique()
all_teams = [t for t in all_teams if pd.notna(t) and t not in ['2TM', '3TM', 'FA', 'N/A - Rookie']]


# Calculate Grades
print("\n" + "="*80)
print("STEP 3: Calculating Offseason Grades with Enhanced Role Fit")
print("="*80)

offseason_grades = []
retention_weight = 0.4

for team in sorted(all_teams):
    gains, losses, retained = identify_transactions(fit_df, team)
    
    total_gains = gains['Weighted Fit'].sum()
    
    # Apply players lost but still free agents a discount
    losses_adjusted = losses.copy()
    losses_adjusted.loc[losses_adjusted['2025-2026 Team'] == 'FA', 'Weighted Fit'] *= 0.5
    total_losses = losses_adjusted['Weighted Fit'].sum()
    
    total_retained = retained['Weighted Fit'].sum()
    
    # Rookie value
    rookie_value = 0.0
    
    net_impact = total_gains - total_losses + (retention_weight * total_retained) + rookie_value
    final_score = net_impact
    
    # Assign grade
    if final_score >= 50:
        grade = 'A+'
    elif final_score >= 35:
        grade = 'A'
    elif final_score >= 25:
        grade = 'A-'
    elif final_score >= 15:
        grade = 'B+'
    elif final_score >= 5:
        grade = 'B'
    elif final_score >= 0:
        grade = 'B-'
    elif final_score >= -5:
        grade = 'C+'
    elif final_score >= -15:
        grade = 'C'
    elif final_score >= -25:
        grade = 'C-'
    elif final_score >= -40:
        grade = 'D'
    else:
        grade = 'F'
    
    offseason_grades.append({
        'Team': team,
        'Grade': grade,
        'Total Gains': total_gains,
        'Total Losses': total_losses,
        'Total Retained': total_retained,
        'Total Rookies': rookie_value,
        'Net Impact': net_impact,
        'Final Score': final_score,
        'Num Gains': len(gains),
        'Num Losses': len(losses),
        'Num Retained': len(retained)
    })
    
    print(f"{team:20s} | Gains: {total_gains:6.2f} | Losses: {total_losses:6.2f} | Retained: {total_retained:6.2f} | Net: {net_impact:6.2f} | Grade: {grade}")

grades_df = pd.DataFrame(offseason_grades).sort_values('Final Score', ascending=False)
grades_df.insert(0, 'Rank', range(1, len(grades_df) + 1))


# Save results
print("\n" + "="*80)
print("Exporting V2 Results...")
print("="*80)

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    grades_df.to_excel(writer, sheet_name='Offseason Grades V2', index=False)
    fit_df.to_excel(writer, sheet_name='Player Fit Scores V2', index=False)

print(f"✓ V2 results exported to: {output_file}")

# Complete
print("\n" + "="*80)
print("Complete!")
print("="*80)
print("\nEnhanced Role Fit Formula:")
print("  • 30% Pace Alignment (tempo match)")
print("  • 30% Portability (versatility)")
print("  • 20% Shooting Alignment (3P style match)")
print("  • 20% Usage Quality (rewards high-usage players)")